## wsi图像数据预处理分析

### 1.统计otsu生成的前景图点数目

In [16]:
import os
import glob

import numpy as np

import openslide
from scipy import ndimage as nd
from random import shuffle

-------------

### 1.统计otsu生成的前景图点数目

In [99]:
otsu_folder = '/root/workspace/dataset/prepare_data/16/wsi_otsu_save/train_resize_64/'
otsu_npy_list = glob.glob(os.path.join(otsu_folder, '*.npy'))
print('total number of npy: %d' % len(otsu_npy_list))

total number of npy: 270


In [100]:
total_points = 0
otsu_points = {}
otsu_npy_list.sort()
for otsu_npy in otsu_npy_list:
    tif_name = otsu_npy.split('_resize')[0]
    otsu_mask = np.load(otsu_npy)
    x, y = np.where(otsu_mask == True)
    otsu_points[tif_name] = len(x)
    total_points += len(x)
print(total_points)

82604427


In [7]:
x,y = np.where(otsu_mask==True)

In [10]:
xy = [[x[i],y[i]] for i in range(len(x))]

In [17]:
shuffle(xy)

In [18]:
distance = nd.distance_transform_edt(otsu_mask)

In [21]:
distance.min()

0.0

In [22]:
Threshold = 75 / (0.243 * 64 * 2)  # 75µm is the equivalent size of 5 tumor cells

In [23]:
Threshold

2.4112654320987654

In [26]:
100000/256

390.625

In [194]:
a = np.array([[1,1,1,1,255],
              [1,0,0,0,1],
              [1,0,0,0,1],
              [1,0,0,1,0],
              [1,1,1,0,1]])

In [195]:
b = np.zeros([5,5,3])
b[:,:,0] = a
b[:,:,1] = a
b[:,:,2] = a

In [196]:
b[:,:,0]

array([[  1.,   1.,   1.,   1., 255.],
       [  1.,   0.,   0.,   0.,   1.],
       [  1.,   0.,   0.,   0.,   1.],
       [  1.,   0.,   0.,   1.,   0.],
       [  1.,   1.,   1.,   0.,   1.]])

In [197]:
distance = nd.distance_transform_edt(b[:, :, 0])

In [201]:
distance

array([[1.41421356, 1.        , 1.        , 1.        , 1.41421356],
       [1.        , 0.        , 0.        , 0.        , 1.        ],
       [1.        , 0.        , 0.        , 0.        , 1.        ],
       [1.        , 0.        , 0.        , 1.        , 0.        ],
       [1.41421356, 1.        , 1.        , 0.        , 1.        ]])

In [202]:
fill_hole = nd.morphology.binary_fill_holes(distance)
fill_hole

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True, False],
       [ True,  True,  True, False,  True]])

In [200]:
from skimage import measure
from scipy import ndimage as nd

In [164]:
evaluation_mask = measure.label(fill_hole, connectivity=2)
evaluation_mask

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 0, 1]])

In [151]:
max_label = np.amax(evaluation_mask)
max_label

1

In [95]:
properties = measure.regionprops(evaluation_mask)
properties

In [98]:
properties[1].major_axis_length

2.3094010767585034

In [77]:
import math
math.sqrt(50)

7.0710678118654755

In [94]:
sorted(set([1,2,3,4,5,6,7,7,7,8,1.2]))

[1, 1.2, 2, 3, 4, 5, 6, 7, 8]